In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP

from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

import atac_utils

In [ ]:
condcode2cond = atac_utils.CONDCODE_TO_COND
# condcodes = np.sort(list(condcode2cond.keys()))
# condcodes
condcode2cond

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21_NRDR.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
%%time
adatas_pk = []
for cond_code, cond_name in condcode2cond.items():
    f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/results_atac/peak_mat_{cond_name}.h5ad'
    adata_pk = sc.read(f)
    adatas_pk.append(adata_pk)
    print(adata_pk)
    # break

In [ ]:
# get cells based on RNA
# get matrix based on ATAC - CPM or CP10k

SHUFFLE = False

# offset = 1e-10
offset = 1

qs_ca_alltime = []
qs_ba_alltime = []
qs_bc_alltime = []

l2fc_ca_alltime = []
l2fc_ba_alltime = []
l2fc_bc_alltime = []

for cond_code, cond_name in condcode2cond.items():
    # get sub
    adatasub = adata[adata.obs['cond']==cond_name]
    adata_pk = adatas_pk[cond_code]
    
    # get A vs C 
    x0 = adatasub.obsm['pca_p17on'][...,0]
    # x_shuff
    if SHUFFLE:
        x = x0[np.random.choice(len(x0), size=len(x0), replace=False)]
    else:
        x = x0
        
    cond_a = x < np.percentile(x,20)
    cond_b = np.logical_and(
                    x > np.percentile(x,40),
                    x < np.percentile(x,60),)
    cond_c = x > np.percentile(x,80)
    cells_a = adatasub[cond_a].obs.index.values
    cells_b = adatasub[cond_b].obs.index.values
    cells_c = adatasub[cond_c].obs.index.values
    
    cells_atac = adata_pk.obs.index.values
    adata_pk_a = adata_pk[np.intersect1d(cells_a, cells_atac)]
    adata_pk_b = adata_pk[np.intersect1d(cells_b, cells_atac)]
    adata_pk_c = adata_pk[np.intersect1d(cells_c, cells_atac)]
    
    # TODO: peak size (500kb) fix
    mat_a = np.array(adata_pk_a.X.todense()) 
    mat_b = np.array(adata_pk_b.X.todense()) 
    mat_c = np.array(adata_pk_c.X.todense()) 
    
    # size norm (CPM) between cells
    mat_a = mat_a/np.sum(mat_a, axis=1).reshape(-1,1)*1e6
    mat_b = mat_b/np.sum(mat_b, axis=1).reshape(-1,1)*1e6
    mat_c = mat_c/np.sum(mat_c, axis=1).reshape(-1,1)*1e6
    
    # DEGs
    lfc_ca = np.log2((np.mean(mat_c, axis=0)+offset)/(np.mean(mat_a, axis=0)+offset))
    lfc_ba = np.log2((np.mean(mat_b, axis=0)+offset)/(np.mean(mat_a, axis=0)+offset))
    lfc_bc = np.log2((np.mean(mat_b, axis=0)+offset)/(np.mean(mat_c, axis=0)+offset))
    
    ts, ps_ca = stats.ttest_ind(mat_c, mat_a)
    ts, ps_ba = stats.ttest_ind(mat_b, mat_a)
    ts, ps_bc = stats.ttest_ind(mat_b, mat_c)
    
    rs, qs_ca, _, _ = multipletests(np.nan_to_num(ps_ca, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_ba, _, _ = multipletests(np.nan_to_num(ps_ba, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_bc, _, _ = multipletests(np.nan_to_num(ps_bc, nan=1).reshape(-1,), method='fdr_bh')
    
    # sig_cond
    cond_sig_a = np.all([qs_ca < 0.05, lfc_ca < -np.log2(2)], axis=0)
    cond_sig_c = np.all([qs_ca < 0.05, lfc_ca >  np.log2(2)], axis=0)
    cond_sig_b  = np.all([np.minimum(qs_ba, qs_bc) < 0.05, 
                          np.mean([lfc_ba, lfc_bc], axis=0) > np.log2(2),
                          lfc_ba > 0, lfc_bc > 0,], axis=0)
    
    print(cond_code, cond_name, len(mat_a), len(mat_b), len(mat_c), 
          cond_sig_a.sum(), cond_sig_c.sum(), cond_sig_b.sum())
    
    # save this
    qs_ca_alltime.append(qs_ca)
    qs_ba_alltime.append(qs_ba)
    qs_bc_alltime.append(qs_bc)
    
    l2fc_ca_alltime.append(lfc_ca)
    l2fc_ba_alltime.append(lfc_ba)
    l2fc_bc_alltime.append(lfc_bc)
    
    # break
qs_ca_alltime = np.array(qs_ca_alltime)
qs_ba_alltime = np.array(qs_ba_alltime)
qs_bc_alltime = np.array(qs_bc_alltime)

l2fc_ca_alltime = np.array(l2fc_ca_alltime)
l2fc_ba_alltime = np.array(l2fc_ba_alltime)
l2fc_bc_alltime = np.array(l2fc_bc_alltime)

In [ ]:
%%time

fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_allconds_ca.txt'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_allconds_ca.txt'
np.savetxt(fout1, qs_ca_alltime)
np.savetxt(fout2, l2fc_ca_alltime)

fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_allconds_ba.txt'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_allconds_ba.txt'
np.savetxt(fout1, qs_ba_alltime)
np.savetxt(fout2, l2fc_ba_alltime)

fout1 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/qs_allconds_bc.txt'
fout2 = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/l2fc_allconds_bc.txt'
np.savetxt(fout1, qs_bc_alltime)
np.savetxt(fout2, l2fc_bc_alltime)

In [ ]:
# get cells based on RNA
# get matrix based on ATAC - CPM or CP10k

SHUFFLE = True

# offset = 1e-10
offset = 1

qs_ca_alltime = []
qs_ba_alltime = []
qs_bc_alltime = []

l2fc_ca_alltime = []
l2fc_ba_alltime = []
l2fc_bc_alltime = []

for cond_code, cond_name in condcode2cond.items():
    # get sub
    adatasub = adata[adata.obs['cond']==cond_name]
    adata_pk = adatas_pk[cond_code]
    
    # get A vs C 
    x0 = adatasub.obsm['pca_p17on'][...,0]
    # x_shuff
    if SHUFFLE:
        x = x0[np.random.choice(len(x0), size=len(x0), replace=False)]
    else:
        x = x0
        
    cond_a = x < np.percentile(x,20)
    cond_b = np.logical_and(
                    x > np.percentile(x,40),
                    x < np.percentile(x,60),)
    cond_c = x > np.percentile(x,80)
    cells_a = adatasub[cond_a].obs.index.values
    cells_b = adatasub[cond_b].obs.index.values
    cells_c = adatasub[cond_c].obs.index.values
    
    cells_atac = adata_pk.obs.index.values
    adata_pk_a = adata_pk[np.intersect1d(cells_a, cells_atac)]
    adata_pk_b = adata_pk[np.intersect1d(cells_b, cells_atac)]
    adata_pk_c = adata_pk[np.intersect1d(cells_c, cells_atac)]
    
    # TODO: peak size (500kb) fix
    mat_a = np.array(adata_pk_a.X.todense()) 
    mat_b = np.array(adata_pk_b.X.todense()) 
    mat_c = np.array(adata_pk_c.X.todense()) 
    
    # size norm (CPM) between cells
    mat_a = mat_a/np.sum(mat_a, axis=1).reshape(-1,1)*1e6
    mat_b = mat_b/np.sum(mat_b, axis=1).reshape(-1,1)*1e6
    mat_c = mat_c/np.sum(mat_c, axis=1).reshape(-1,1)*1e6
    
    # DEGs
    lfc_ca = np.log2((np.mean(mat_c, axis=0)+offset)/(np.mean(mat_a, axis=0)+offset))
    lfc_ba = np.log2((np.mean(mat_b, axis=0)+offset)/(np.mean(mat_a, axis=0)+offset))
    lfc_bc = np.log2((np.mean(mat_b, axis=0)+offset)/(np.mean(mat_c, axis=0)+offset))
    
    ts, ps_ca = stats.ttest_ind(mat_c, mat_a)
    ts, ps_ba = stats.ttest_ind(mat_b, mat_a)
    ts, ps_bc = stats.ttest_ind(mat_b, mat_c)
    
    rs, qs_ca, _, _ = multipletests(np.nan_to_num(ps_ca, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_ba, _, _ = multipletests(np.nan_to_num(ps_ba, nan=1).reshape(-1,), method='fdr_bh')
    rs, qs_bc, _, _ = multipletests(np.nan_to_num(ps_bc, nan=1).reshape(-1,), method='fdr_bh')
    
    # sig_cond
    cond_sig_a = np.all([qs_ca < 0.05, lfc_ca < -np.log2(2)], axis=0)
    cond_sig_c = np.all([qs_ca < 0.05, lfc_ca >  np.log2(2)], axis=0)
    cond_sig_b  = np.all([np.minimum(qs_ba, qs_bc) < 0.05, 
                          np.mean([lfc_ba, lfc_bc], axis=0) > np.log2(2),
                          lfc_ba > 0, lfc_bc > 0,], axis=0)
    
    print(cond_code, cond_name, len(mat_a), len(mat_b), len(mat_c), 
          cond_sig_a.sum(), cond_sig_c.sum(), cond_sig_b.sum())
    
    # save this
    qs_ca_alltime.append(qs_ca)
    qs_ba_alltime.append(qs_ba)
    qs_bc_alltime.append(qs_bc)
    
    l2fc_ca_alltime.append(lfc_ca)
    l2fc_ba_alltime.append(lfc_ba)
    l2fc_bc_alltime.append(lfc_bc)
    
    # break
qs_ca_alltime = np.array(qs_ca_alltime)
qs_ba_alltime = np.array(qs_ba_alltime)
qs_bc_alltime = np.array(qs_bc_alltime)

l2fc_ca_alltime = np.array(l2fc_ca_alltime)
l2fc_ba_alltime = np.array(l2fc_ba_alltime)
l2fc_bc_alltime = np.array(l2fc_bc_alltime)